In [ ]:
# %load bias_stability.py
#!/usr/bin/python

# Plots stability of a set of bias frames vs time

import matplotlib
matplotlib.use("PDF")
import pyfits as pf
from pylab import *
import sys, glob, time

thedir='/Users/cslage/Research/LSST/code/GUI/flats/20160107_varying_flux/'
%cd $thedir

keys = ['SEGMENT10','SEGMENT11','SEGMENT12','SEGMENT13','SEGMENT14','SEGMENT15',
        'SEGMENT16','SEGMENT17','SEGMENT07','SEGMENT06','SEGMENT05','SEGMENT04',
        'SEGMENT03','SEGMENT02','SEGMENT01','SEGMENT00']
values = range(16)
segdict = dict(zip(keys, values))
#%matplotlib inline

In [ ]:
infilename = "correlations.txt"
covsteps = 6
numsegments = 16
file = open(infilename,'r')
lines = file.readlines()
file.close
numfiles = int(floor(len(lines) / (15*36)))
covariance = zeros([covsteps, covsteps, numsegments, numfiles])
outfilename = thedir+'Correlations_%d.pdf'%numfiles
for line in lines:
    items = line.split()
    if items[0] == 'ii':
        continue
    try:
        ii = int(items[0])
        jj = int(items[1])
        n = int(items[2])
        segment = segdict[items[3]]
        covariance[ii,jj,segment,n] = float(items[4]) 
        #if ii == 0 and jj == 0 and (float(items[4]) > 7000.0 or float(items[4]) < 5000.0):
        #    print n, float(items[4])
    except:
        break
    
x = []
y = []
yerr = []
xneg = []
yneg = []
yerr_neg = []
xfit = []
yfit = []
variance = 0.0
numvar = 0
for ii in range(covsteps):
    for jj in range(covsteps):
        print "ii = %d, jj = %d, Cij = %.4f, Cij-s = %.4f"%\
        (ii,jj,covariance[ii,jj,0:15,:].mean(),covariance[ii,jj,0:15,:].std())
        rsquared = ii**2 + jj**2
        yvalue = covariance[ii,jj,0:15,:].mean()
        if rsquared == 0:
            variance += yvalue
            numvar += 1
        elif yvalue < 0:
            xneg.append(rsquared)
            yneg.append(-yvalue)
            yerr_neg.append(covariance[ii,jj,0:15,:].std())
        else:
            x.append(rsquared)
            y.append(yvalue)
            yerr.append(covariance[ii,jj,0:15,:].std())
        if rsquared > 1.1 and rsquared < 18 and yvalue > 0:
            xfit.append(rsquared)
            yfit.append(yvalue)
variance /= float(numvar)
num_meas = numfiles * (numsegments - 1)
y /= variance
yerr /= (variance * sqrt(float(num_meas)))
yneg /= variance
yerr_neg /= (variance * sqrt(float(num_meas)))
yfit /= variance
ylower = np.maximum(2E-5, y - yerr)
yerr_lower = y - ylower
yneg_lower = np.maximum(2E-5, yneg - yerr_neg)
yerr_neg_lower = yneg - yneg_lower
print "Average Variance = %f"%variance
figure()
title("Correlation Coefficient %d Pairs of Flats"%numfiles)
xscale('log')
yscale('log')
xlim(0.8,100.0)
ylim(1.0E-5,1.0E-1)
errorbar(x,y, yerr = [yerr_lower, 2*yerr] , ls = 'None',marker = '.', ms = 10, color = 'blue')
errorbar(xneg,yneg, yerr = [yerr_neg_lower, 2*yerr_neg], ls = 'None', marker = '.',ms = 10,color = 'magenta', label = 'Negative Values')
#errorbar(x,y, yerr = [0.0*yerr_lower, 0.0**yerr] , ls = 'None',marker = 'o')
#errorbar(xneg,yneg, yerr = [0.0*yerr_neg_lower, 0.0**yerr_neg], ls = 'None', marker = 'o',color = 'magenta', label = 'Negative Values')
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(log10(xfit),log10(yfit))
xplot=linspace(0.0, 2.0, 100)
yplot = slope * xplot + intercept
plot(10**xplot, 10**yplot, color='red', lw = 2, ls = '--')
text(10.0, 0.005, "Slope = %.3f"%slope)
xlabel("$i^2 + j^2$")
ylabel("Covariance")
legend()
#show()
savefig(outfilename)
#close("All")


In [ ]:
13000*5.6

In [ ]:
x=[20000.0,40000.0,60000.0,80000.0,100000.0]
y = [.0052,.0094,.0133,.0177,.0219]
scatter(x,y)
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
xplot=linspace(0.0, 120000.0, 100)
yplot = slope * xplot + intercept
print intercept
plot(xplot,yplot,color='red')
xlim(0.0,120000.0)
xlabel("Flux(e-)")
ylabel("Covariance")
show()